In [1]:
import torch
torch.cuda.is_available()

True

In [3]:
# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")

CUDA is available. Using GPU.


In [12]:
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [13]:
# Load data
data_path = "C:/Users/hanba/OneDrive/Documents/GitProjects/MLHWCore/Homework/Homework 6 dev Neural Network/data/Housing-valid.csv"
data = pd.read_csv(data_path)

In [17]:
### Problem 1

In [14]:
# Assuming the last column is the target and the rest are features
X = data.iloc[:, :-1]  # All columns except the last one
y = data.iloc[:, -1]   # The last column

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert to PyTorch tensors
X = torch.tensor(X_scaled).float()
y = torch.tensor(y.values).float().unsqueeze(1)  # Adding an extra dimension to y for compatibility

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create data loaders
train_data = TensorDataset(X_train, y_train)
val_data = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64)

# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.fc2 = nn.Linear(32, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Determine if GPU is available and move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleNN(input_size=X_train.shape[1]).to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10  # Set the number of epochs
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation step
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs, targets).item()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss / len(val_loader)}")

# Model complexity
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters: ", total_params)

Epoch 1, Loss: 0.21004599332809448, Validation Loss: 0.5005009770393372
Epoch 2, Loss: 0.4412330090999603, Validation Loss: 0.4693121016025543
Epoch 3, Loss: 0.34730014204978943, Validation Loss: 0.4405355155467987
Epoch 4, Loss: 0.1968231201171875, Validation Loss: 0.41411200165748596
Epoch 5, Loss: 0.30807992815971375, Validation Loss: 0.3896062672138214
Epoch 6, Loss: 0.20633237063884735, Validation Loss: 0.3662286102771759
Epoch 7, Loss: 0.18456165492534637, Validation Loss: 0.34424516558647156
Epoch 8, Loss: 0.1640472263097763, Validation Loss: 0.32400497794151306
Epoch 9, Loss: 0.1554933488368988, Validation Loss: 0.305120050907135
Epoch 10, Loss: 0.17528299987316132, Validation Loss: 0.28770652413368225
Total number of parameters:  449


In [ ]:
#B
